In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import requests
from plotly.subplots import make_subplots

In [ ]:
df=pd.read_parquet('dataset_full.parquet')
df.head()

# Vérification rapide
print(f"Dataset chargé : {df.shape[0]} lignes, {df.shape[1]} colonnes")
df.head()

In [ ]:
df.dtypes

---

#### Vérifications finales avant EDA

In [ ]:
# Vérification d'éventuels Nan

# Calculer le pourcentage de NaN par colonne
nan_percent = df.isna().mean() * 100

# Sélectionner seulement les colonnes avec au moins un NaN
nan_columns = nan_percent[nan_percent > 0].sort_values(ascending=False)

# Afficher
print(nan_columns)

---

#### A. L'AGE

##### 1. Comparatif ouvertes vs fermées

In [ ]:
# 1. Préparation minimale : on mappe juste les 0/1 pour la légende
df_plot = df.assign(Statut = df["fermeture"].map({0: "Ouvertes", 1: "Fermées"}))

# 2. Création du graphique en une ligne
fig = px.histogram(
    df_plot, 
    x="age_estime", 
    color="Statut",
    barmode="group",
    color_discrete_map={"Ouvertes": "green", "Fermées": "red"},
    category_orders={"Statut": ["Ouvertes", "Fermées"]},
    title="Comparatif sociétés ouvertes vs fermées selon l'âge"
)

# 3. Ajustements esthétiques
fig.update_layout(
    xaxis_title="Âge estimé (ans)",
    yaxis_title="Nombre de sociétés",
    xaxis=dict(tickmode='linear'),
    template='plotly_white',
    width=1300,
    height=600
)

fig.show()


#### Probabilité de fermeture selon l'age

In [ ]:
df_age_events = (
    df
    .loc[df["age_estime"].ge(0)]
    .assign(age_estime=lambda x: x["age_estime"].astype(int))
    .groupby("age_estime")["fermeture"]
    .agg(fermetures="sum", observations="count")
    .assign(proba_fermeture=lambda x: x["fermetures"] / x["observations"])
    .reset_index()
)

df_age_events = df_age_events[df_age_events["age_estime"] <= 35]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_age_events["age_estime"],
    y=df_age_events["proba_fermeture"] * 100,
    mode="lines+markers",
    line=dict(width=3),
    marker=dict(size=6)
))


fig.update_layout(
    title="Probabilité empirique de fermeture selon l’âge",
    xaxis_title="Années",
    template="plotly_white",
    width=1200,
    height=500
)

fig.show()

Les données montrent une mortalité extrêmement élevée dans les 3 premières années.

À partir de 6 ans, les fermetures remontent.

---

#### Centrage sur les valeurs la plage la plus importante à partir de janvier 2023 jusqu'en septembre 2025

In [ ]:
# 1. Pipeline de données : Filtrage -> Agrégation -> Conversion -> Filtrage temporel
df_trend = (
    df[(df["fermeture"] == 1) & (df["Date_fermeture_finale"].notna())]
    .assign(Mois = lambda x: x["Date_fermeture_finale"].dt.to_period('M').dt.to_timestamp())
    .groupby("Mois").size().reset_index(name="Nb_Fermetures")
    .query("'2023-01-01' <= Mois <= '2025-09-01'")
)

# 2. Création du graphique avec Plotly Express
fig = px.bar(
    df_trend, 
    x="Mois", 
    y="Nb_Fermetures",
    title="Évolution mensuelle des fermetures (Janvier 2023 - Septembre 2025)",
    template='plotly_white',
    color_discrete_sequence=['darkred']
)

# 3. Ajustements du layout
fig.update_layout(
    width=1200, 
    height=600,
    xaxis_title=None, # Le label "Mois" est explicite par les dates
    yaxis_title="Nombre de sociétés"
)

fig.show()


##### Comparatif mensuel

In [ ]:
# 1. Pipeline de données
df_pivot = (
    df[(df["fermeture"] == 1) & (df["Date_fermeture_finale"].notna())]
    .assign(Année=df["Date_fermeture_finale"].dt.year, 
            Mois=df["Date_fermeture_finale"].dt.month)
    .query("Année >= 2023")
    .pivot_table(index="Mois", columns="Année", values="fermeture", aggfunc="count", fill_value=0)
)

# 2. Calcul des pourcentages (variation annuelle)
df_pct = df_pivot.pct_change(axis=1).fillna(0) * 100

# 3. Graphique avec itération optimisée
mois_labels = ["Jan", "Fév", "Mar", "Avr", "Mai", "Juin", "Juil", "Août", "Sep", "Oct", "Nov", "Déc"]

fig = go.Figure()

for year in df_pivot.columns:

    labels = [f"{v:+.1f}%" if year != df_pivot.columns[0] else "" for v in df_pct[year]]
    
    fig.add_trace(go.Bar(
        x=mois_labels,
        y=df_pivot[year],
        name=str(year),
        text=labels,
        textposition='outside'
    ))

fig.update_layout(
    title="Analyse Comparative Mensuelle : Volume et Évolution des Fermetures",
    barmode='group',
    template='plotly_white',
    width=1500,
    height=600,
    margin=dict(t=80)
)

fig.show()

---

### B. Les secteurs d'activité

In [ ]:
# Filtrer uniquement les sociétés fermées
df_closed = df[df["fermeture"] == 1]

# Compter les fermetures par code APE
ape_counts = (
    df_closed["code_ape"]
    .value_counts()
    .reset_index()
)

ape_counts.columns = ["code_ape", "nb_fermetures"]

# Voir les plus représentés
ape_counts.head(10)

In [ ]:
# 1. Préparation des données en un seul bloc

top_ape = (
    ape_counts.head(10)
    .merge(df[['code_ape', 'libelle_section_ape']].drop_duplicates(), on='code_ape', how='left')
    .assign(label = lambda x: x["code_ape"].astype(str) + " – " + x["libelle_section_ape"])
)

# 2. Création du graphique
fig = px.bar(
    top_ape,
    x="nb_fermetures",
    y="label",
    orientation='h',
    text="nb_fermetures",
    title="Top 10 des secteurs par volume de fermetures",
    template='plotly_white',
    color_discrete_sequence=['firebrick']
)

# 3. Ajustements
# 1. On récupère la valeur max pour calculer l'échelle
val_max = top_ape["nb_fermetures"].max()

# 2. Ajustements
fig.update_traces(
    textposition='outside',
    cliponaxis=False 
)

fig.update_layout(
    height=500,
    xaxis=dict(range=[10000, val_max * 1.15]), 
    xaxis_title="Nombre de fermetures",
    yaxis_title=None,
    margin=dict(l=300, r=80),
    template='plotly_white'
)

fig.update_yaxes(autorange="reversed")

In [ ]:
# 1. Sélectionner les secteurs avec suffisamment d'entreprises (ex: Top 5)
top_secteurs = df['libelle_section_ape'].value_counts().head(5).index.tolist()

fig = go.Figure()

for secteur in top_secteurs:
    # Filtrer et calculer la proba par âge pour ce secteur précis
    df_secteur = (
        df[df["libelle_section_ape"] == secteur]
        .loc[df["age_estime"].between(0, 35)]
        .groupby("age_estime")["fermeture"]
        .agg(fermetures="sum", obs="count")
        .assign(proba=lambda x: (x["fermetures"] / x["obs"]) * 100)
        .reset_index()
    )
    
    # Ajouter la ligne au graphique
    fig.add_trace(go.Scatter(
        x=df_secteur["age_estime"],
        y=df_secteur["proba"],
        mode="lines",
        name=secteur,
        line=dict(width=2),
        hovertemplate="Âge: %{x} ans<br>Risque: %{y:.1f}%<extra></extra>"
    ))

fig.update_layout(
    title="Comparatif du risque de fermeture par secteur d'activité",
    xaxis_title="Âge de l'entreprise (années)",
    yaxis_title="Probabilité de fermeture (%)",
    template="plotly_white",
    hovermode="x unified",
    legend=dict(orientation="h", y=-0.2),
    width=1300,
    height=700
)

fig.show()

In [ ]:
# 1. Identifier les 10 secteurs ayant le plus de fermetures au total
top_10_secteurs = (
    df[df["fermeture"] == 1]["libelle_section_ape"]
    .value_counts()
    .head(10)
    .index.tolist()
)

# 2. Filtrer le dataset sur ces 10 secteurs et sur l'année 2024
df_heatmap_top = (
    df[
        (df["fermeture"] == 1) & 
        (df["libelle_section_ape"].isin(top_10_secteurs)) &
        (df["Date_fermeture_finale"].dt.year == 2024)
    ]
    .assign(Mois = df["Date_fermeture_finale"].dt.month)
    .groupby(["libelle_section_ape", "Mois"])
    .size()
    .reset_index(name="Nb_Fermetures")
)

# 3. Pivot pour la Heatmap
df_pivot_top = df_heatmap_top.pivot(
    index="libelle_section_ape", 
    columns="Mois", 
    values="Nb_Fermetures"
).fillna(0)

# 4. Affichage
fig = px.imshow(
    df_pivot_top,
    labels=dict(x="Mois", y="Secteur", color="Fermetures"),
    x=mois_labels, 
    title="Saisonnalité des fermetures : Focus sur les 10 secteurs les plus touchés (2024)",
    color_continuous_scale="YlOrRd",
    aspect="auto",
    text_auto=True
)

fig.update_layout(width=1100, height=600)
fig.show()

---

#### Proportion des fermetures par secteur

In [ ]:
# 1. Pipeline de données
top10_risk = (
    df.groupby("code_ape")
    .agg(
        total_societes=("fermeture", "size"),
        taux_fermeture=("fermeture", "mean"),
        libellé=("libelle_section_ape", "first")
    )
    .reset_index()
    .query("total_societes >= 500")
    .assign(
        taux_pct = lambda x: (x["taux_fermeture"] * 100).round(1),
        label = lambda x: x["code_ape"].astype(str) + " – " + x["libellé"]
    )
    .sort_values("taux_fermeture", ascending=False)
    .head(10)
)

# 2. Création du graphique avec Plotly Express
fig = px.bar(
    top10_risk,
    x="taux_pct",
    y="label",
    orientation="h",
    text="taux_pct",
    title="Secteurs avec les taux de fermeture les plus élevés (Min. 500 sociétés)",
    template="plotly_white",
    color_discrete_sequence=["darkred"]
)

# 3. Ajustements de mise en page
fig.update_traces(
    texttemplate='%{text}%', 
    textposition='outside',
    cliponaxis=False
)

# On définit une marge de sécurité sur l'axe X (ex: 20% de plus que le max)
valeur_max = top10_risk["taux_pct"].max()

fig.update_layout(
    height=600,
    xaxis_range=[30, valeur_max * 1.2],
    xaxis_title="Taux de fermeture (%)",
    yaxis_title=None,
    margin=dict(l=300, r=60),
    template="plotly_white"
)
fig.update_yaxes(autorange="reversed")

#### Comparaison 2023-2024

In [ ]:
# 1. Préparation des indicateurs annuels
df_work = df.assign(
    annee = df["Date_fermeture_finale"].dt.year,
    is_2023 = lambda x: (x["fermeture"] == 1) & (x["annee"] == 2023),
    is_2024 = lambda x: (x["fermeture"] == 1) & (x["annee"] == 2024)
)

# 2. Agrégation groupée
top10_degradation = (
    df_work.groupby("code_ape")
    .agg(
        total_societes=("fermeture", "size"),
        fermetures_2023=("is_2023", "sum"),
        fermetures_2024=("is_2024", "sum"),
        libellé=("libelle_section_ape", "first")
    )
    .reset_index()
)

# 3. Calculs et filtrage final
top10_degradation = (
    top10_degradation[top10_degradation["total_societes"] >= 500]
    .assign(
        taux_2023 = lambda x: x["fermetures_2023"] / x["total_societes"],
        taux_2024 = lambda x: x["fermetures_2024"] / x["total_societes"],
        evolution_pts = lambda x: ((x["taux_2024"] - x["taux_2023"]) * 100).round(2),
        label = lambda x: x["code_ape"].astype(str) + " – " + x["libellé"]
    )
    .sort_values("evolution_pts", ascending=False)
    .head(10)
)

# 4. Création du graphique avec Plotly Express
fig = px.bar(
    top10_degradation,
    x="evolution_pts",
    y="label",
    orientation="h",
    text="evolution_pts",
    title="Top 10 Dégradations : Hausse du taux de fermeture (2024 vs 2023)",
    template="plotly_white",
    color_discrete_sequence=["firebrick"]
)

# 5. Ajustements finaux
fig.update_traces(
    texttemplate='%{text}+ pts', 
    textposition='outside'
)

max_val = top10_degradation["evolution_pts"].max()

fig.update_layout(
    yaxis_title=None,
    xaxis_title=None,
    height=600,
    margin=dict(l=300),
    xaxis=dict(range=[5, max_val * 1.10])
)

fig.update_yaxes(autorange="reversed")

fig.show()

---

#### Heatmap age / secteur

In [ ]:
# 1. Identifier les 10 secteurs les plus représentés
top_secteurs = df['libelle_section_ape'].value_counts().head(10).index

# 2. Pipeline de traitement
df_heatmap = (
    df[
        (df['libelle_section_ape'].isin(top_secteurs)) & 
        (df['age_estime'] <= 27)
    ]
    .groupby(["libelle_section_ape", "age_estime"])["fermeture"]
    .mean()
    .unstack(fill_value=0)
    .round(2)
)

# 3. Création de la Heatmap avec affichage décimal
fig = px.imshow(
    df_heatmap,
    labels=dict(x="Âge estimé", color="Probabilité"),
    color_continuous_scale="Reds",
    aspect="auto",
    text_auto=".2f",
    title="Probabilités de Fermeture selon les secteurs les plus touchés (Échelle 0-1)"
)

fig.update_layout(
    width=1600, 
    height=750,
    yaxis_title=None,
    xaxis_tickmode='linear',
    coloraxis_colorbar=dict(title="Proba")
)

fig.show()

---

##### 2. Répartition selon formes juridiques

In [ ]:
# 1. Configuration des couleurs fixes
mapping = {5499: "SARL", 5710: "SAS"}
color_map = {"SARL": "blue", "SAS": "orange"}

# 2. Préparation des données

def get_counts(filter_condition=None):
    base = df[df["Catégorie juridique de l'unité légale"].isin([5499, 5710])]
    if filter_condition is not None:
        base = base[base["fermeture"] == filter_condition]
    counts = base["Catégorie juridique de l'unité légale"].map(mapping).value_counts()

    return counts.sort_index()

data_list = [get_counts(), get_counts(0), get_counts(1)]
titles = ["Répartition Totale", "Sociétés Ouvertes", "Sociétés Fermées"]

# 3. Création de la figure
fig = make_subplots(
    rows=1, cols=3, 
    specs=[[{'type':'domain'}]*3],
    subplot_titles=titles
)

for i, data in enumerate(data_list, 1):
    fig.add_trace(
        go.Pie(
            labels=data.index, 
            values=data.values, 
            marker=dict(colors=[color_map[label] for label in data.index]),
            textinfo='percent+label',
        ), 
        row=1, col=i
    )

fig.update_layout(
    title_text="Comparaison Structurelle SARL vs SAS",
    width=1100,
    height=450,
    showlegend=False
)

fig.show()

#### ZOOM ESS

In [ ]:
# 1. Préparation des données (Résilience)
df_ess = (
    df.groupby('Economie sociale et solidaire unité légale')
    .agg(taux_fermeture_brut=('fermeture', 'mean'))
    .reset_index()
)
df_ess['taux_resilience'] = ((1 - df_ess['taux_fermeture_brut']) * 100).round(2)

# 2. Création du graphique
fig = px.bar(
    df_ess, 
    x='Economie sociale et solidaire unité légale', 
    y='taux_resilience', 
    color='Economie sociale et solidaire unité légale',
    text='taux_resilience',
    title="ESS vs Classique : Indice de Résilience",
    labels={'Economie sociale et solidaire unité légale': 'Appartenance à l\'ESS'},
    color_discrete_map={'O': '#2ecc71', 'N': '#e74c3c'}
)

# 3. Ajustements des traces
fig.update_traces(
    texttemplate='%{text}%', 
    textposition='outside'
)

# 4. On nettoie l'axe Y original et on règle les marges
fig.update_yaxes(
    title_text="",
    range=[df_ess['taux_resilience'].min() - 5, df_ess['taux_resilience'].max() + 5]
)

fig.update_layout(
    width=700, # Ta nouvelle largeur
    height=500,
    showlegend=False,
    margin=dict(t=80, b=50, l=120, r=50),
    
    # On ajoute le titre horizontalement via une annotation
    annotations=[dict(
        x=-0.15,
        y=0.5,
        xref="paper",
        yref="paper",
        text="Taux de<br>Survie ",
        showarrow=False,
        font=dict(size=13, color="black"),
        align="center"
    )]
)

fig.show()

---

### C. L'EFFECTIF

In [ ]:
# 1. Préparation des données

order = sorted(df["Tranche_effectif_num"].unique())

def get_effectif_counts(filter_condition=None):
    base = df.copy()
    if filter_condition is not None:
        base = base[base["fermeture"] == filter_condition]
    
    # Comptage par tranche
    counts = base["Tranche_effectif_num"].value_counts().reindex(order, fill_value=0)
    return counts

data_list = [get_effectif_counts(), get_effectif_counts(0), get_effectif_counts(1)]
titles = ["Effectifs : Répartition Totale", "Sociétés Ouvertes", "Sociétés Fermées"]

# 2. Création de la figure
fig = make_subplots(
    rows=1, cols=3, 
    specs=[[{'type':'domain'}]*3],
    subplot_titles=titles
)

# Utilisation d'une palette de couleurs séquentielle pour marquer la progression de la taille
colors = px.colors.sequential.Blues_r

for i, data in enumerate(data_list, 1):
    fig.add_trace(
        go.Pie(
            labels=data.index.astype(str),
            values=data.values, 
            marker=dict(colors=colors),
            textinfo='percent',
            hole=0.4,
            name=""
        ), 
        row=1, col=i
    )

# 3. Layout
fig.update_layout(
    title_text="Analyse par Taille d'Entreprise (Tranches d'effectifs)",
    width=1200,
    height=500,
    legend_title="Tranches d'effectifs",
    legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="center", x=0.5)
)

fig.show()

In [ ]:
# 1. Calcul du taux de fermeture par tranche d'effectif
df_size_risk = (
    df.groupby("Tranche_effectif_num")
    .agg(
        total=("fermeture", "size"),
        nb_fermes=("fermeture", "sum")
    )
    .reset_index()
)

df_size_risk["taux_fermeture"] = (df_size_risk["nb_fermes"] / df_size_risk["total"] * 100).round(2)

# 2. Visualisation
fig = px.bar(
    df_size_risk,
    x="Tranche_effectif_num",
    y="taux_fermeture",
    text="taux_fermeture",
    title="Lien entre Taille de l'entreprise et Risque de fermeture",
    labels={
        "Tranche_effectif_num": "Tranche d'effectif (Codes)",
    },
    color="taux_fermeture",
    color_continuous_scale="Reds"
)

# On calcule la valeur max pour ajuster l'échelle
max_taux = df_size_risk["taux_fermeture"].max()

fig.update_traces(
    texttemplate='%{text}%', 
    textposition='outside',
    cliponaxis=False
)

fig.update_layout(
    xaxis_type='category',
    yaxis=dict(
        range=[0, max_taux * 1.15],
        title={'text': ''}
    ), 
    width=900,  
    height=600,
    margin=dict(t=80, b=50, l=60, r=50)
)

fig.show()

---

### D. LA GEOGRAPHIE

In [ ]:
# 1. Récupération du tracé des départements (GeoJSON)

repo_url = "https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements-version-simplifiee.geojson"
geojson_france = requests.get(repo_url).json()

# 2. Préparation des données : Calcul du taux de fermeture par département
df_dept = (
    df.groupby("Code du département de l'établissement")
    .agg(
        total=("fermeture", "size"),
        taux_fermeture=("fermeture", "mean")
    )
    .reset_index()
)

# On convertit le taux en pourcentage pour la lisibilité
df_dept["taux_pct"] = (df_dept["taux_fermeture"] * 100).round(2)

moyenne_nationale = df_dept["taux_pct"].mean()

# 2. Création de la carte nuancée
fig = px.choropleth(
    df_dept,
    geojson=geojson_france,
    locations="Code du département de l'établissement",
    featureidkey="properties.code",
    color="taux_pct",
    color_continuous_scale="RdBu_r", 
    color_continuous_midpoint=moyenne_nationale,
    scope="europe",
    title=f"Indice de sinistralité géographique (Moyenne : {moyenne_nationale:.2f}%)",
    labels={'taux_pct': 'Taux de fermeture (%)'}
)

# 3. Optimisation du rendu
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(
    margin={"r":0,"t":80,"l":0,"b":0},
    width=1000,
    height=700,
    coloraxis_colorbar=dict(title="Risque", ticksuffix="%")
)

fig.show()

In [ ]:
# 1. Données géo
geojson_dept = requests.get("https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements-version-simplifiee.geojson").json()

# 2. Calcul du "Taux de Résilience" (Entreprises de + de 10 ans)
df_resilience = (
    df[df["fermeture"] == 0]
    .groupby("Code du département de l'établissement")
    .agg(
        total=("age_estime", "count"),
        plus_de_10ans=("age_estime", lambda x: (x > 10).sum())
    )
    .reset_index()
)
df_resilience["taux_vieux"] = (df_resilience["plus_de_10ans"] / df_resilience["total"] * 100).round(2)

# 3. Création de la carte (SANS TRICHER sur les rangs)
fig = px.choropleth(
    df_resilience,
    geojson=geojson_dept,
    locations="Code du département de l'établissement",
    featureidkey="properties.code",
    color="taux_vieux",
    color_continuous_scale="Cividis_r", 
    scope="europe",
    title="Part des entreprises de plus de 10 ans par département (%)",
    labels={'taux_vieux': '% Entreprises > 10 ans'}
)

# 4. Ajustements de forme
fig.update_geos(fitbounds="locations", visible=False, projection_type="mercator")
fig.update_layout(
    margin={"r":50,"t":80,"l":50,"b":50},
    width=800,
    height=900,
    coloraxis_colorbar=dict(title="Taux (%)", ticksuffix="%")
)

fig.show()

In [ ]:
# 1. GeoJSON des départements
geojson_dept = requests.get("https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements-version-simplifiee.geojson").json()

# 2. Préparation des données (Sociétés FERMÉES par département)
df_map_ferme = (
    df[df["fermeture"] == 1]
    .groupby("Code du département de l'établissement")["age_estime"]
    .mean()
    .reset_index()
    .assign(
        # On divise en 10 groupes pour forcer le contraste visuel
        rang_survie = lambda x: pd.qcut(x["age_estime"], 10, labels=False)
    )
)

# 3. Création de la carte
fig = px.choropleth(
    df_map_ferme,
    geojson=geojson_dept,
    locations="Code du département de l'établissement",
    featureidkey="properties.code",
    color="rang_survie",
    color_continuous_scale="Blues",
    hover_data={"rang_survie": False, "age_estime": ":.2f"},
    title="Durée de vie moyenne avant fermeture par Département (Contraste par déciles)"
)

    # 3. LE RÉGLAGE ANTI-ÉCRASEMENT
fig.update_geos(
    fitbounds="locations", 
    visible=False,
    projection_type='mercator' 
)

fig.update_layout(
    width=800,  
    height=800, 

    margin={"r":50, "t":80, "l":50, "b":50}, 
    
    coloraxis_colorbar=dict(
        title="Durée de vie", 
        tickvals=[0, 9], 
        ticktext=["Courte", "Longue"],
        thickness=20 
    )
)

fig.show()

---

In [ ]:
# 1. GeoJSON
geojson_dept = requests.get("https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/departements-version-simplifiee.geojson").json()

# 2. Préparation : On définit ce qu'est un "Principal Employeur"

seuil_fixe = 10 

df_employeurs = (
    df[df["Tranche_effectif_num"] >= seuil_fixe]
    .groupby("Code du département de l'établissement")
    .size()
    .reset_index(name="nb_gros_employeurs")
)

# 3. Création de la carte (Heatmap de densité)
fig = px.choropleth(
    df_employeurs,
    geojson=geojson_dept,
    locations="Code du département de l'établissement",
    featureidkey="properties.code",
    color="nb_gros_employeurs",
    color_continuous_scale="YlGnBu",
    scope="europe",
    title=f"Concentration des principaux employeurs (Tranche d'effectifs >= {seuil_fixe})",
    labels={'nb_gros_employeurs': 'Nombre d\'entreprises',
            'taux_fermeture':""}
)

# 4. Ajustements anti-écrasement
fig.update_geos(fitbounds="locations", visible=False, projection_type="mercator")
fig.update_layout(
    margin={"r":50,"t":80,"l":50,"b":50},
    width=1000,
    height=800,
    coloraxis_colorbar=dict(title="Volume")
)

fig.show()

---

#### Où sont les départements qui ont à la fois des entreprises solides et qui emploient massivement ?

In [ ]:
# 1. Préparation des données combinées
df_final = df[df["fermeture"] == 0].groupby("Code du département de l'établissement").agg(
    total=("age_estime", "count"),
    plus_de_10ans=("age_estime", lambda x: (x > 10).sum()),
    gros_employeurs=("Tranche_effectif_num", lambda x: (x >= 10).sum())
).reset_index()

df_final["taux_resilience"] = (df_final["plus_de_10ans"] / df_final["total"] * 100).round(2)

# 2. Ajout des coordonnées des centres des départements (approximation pour les bulles)

fig = px.choropleth(
    df_final,
    geojson=geojson_dept,
    locations="Code du département de l'établissement",
    featureidkey="properties.code",
    color="taux_resilience",
    hover_data=["gros_employeurs", "total"],
    color_continuous_scale="Viridis",
    scope="europe",
    title="Synthèse Économique : Résilience (Couleur) et Poids des Employeurs"
)

# 3. Optimisation du rendu
fig.update_geos(fitbounds="locations", visible=False, projection_type="mercator")
fig.update_layout(
    width=800,
    height=900,
    margin={"r":50,"t":80,"l":50,"b":50},
    coloraxis_colorbar=dict(title="Taux de +10 ans (%)")
)

fig.show()

#### Dernier Zoom sur les Bouches-du-Rhône : Age, Effectif et Emplacement

In [ ]:
# --- 1. CALCULS GLOBAUX (Identiques et robustes) ---
df_taux = df.groupby("Code du département de l'établissement")["fermeture"].mean().reset_index()
df_taux.columns = ["Code du département de l'établissement", "taux_pct"]
df_taux["taux_pct"] = (df_taux["taux_pct"] * 100).round(2)

df_survie = df[df["fermeture"] == 1].groupby("Code du département de l'établissement")["age_estime"].mean().reset_index()
df_survie.columns = ["Code du département de l'établissement", "survie_moyenne"]
df_survie["survie_moyenne"] = df_survie["survie_moyenne"].round(1)

df_stats = pd.merge(df_taux, df_survie, on="Code du département de l'établissement", how="left")

# --- 2. CONFIGURATION DU DEPARTEMENT ---
dep_cible = "13"

stats_locales = df_stats[df_stats["Code du département de l'établissement"] == dep_cible].iloc[0]
t_local = stats_locales["taux_pct"]
s_local = stats_locales["survie_moyenne"]

moy_nat_taux = df_stats["taux_pct"].mean().round(2)
moy_nat_survie = df_stats["survie_moyenne"].mean().round(1)

# --- 3. FILTRAGE ET CARTE ---
df_selection = df[df["Code du département de l'établissement"] == dep_cible]

fig = px.scatter_mapbox(
    df_selection,
    lat="latitude",
    lon="longitude",
    color="age_estime",
    size="Tranche_effectif_num",
    color_continuous_scale="Viridis",
    size_max=12,
    hover_name="Dénomination de l'unité légale",
    mapbox_style="carto-positron",
    title=f"<b>Analyse Territoriale - Dept {dep_cible}</b>",
    height=800
)

# --- 4. LA LÉGENDE STABLE (ANNOTATION) ---
fig.update_layout(
    margin={"r":0,"t":60,"l":0,"b":0},
    dragmode="pan",
    annotations=[dict(
        x=0.01, y=0.99,
        xref="paper", yref="paper",
        text=(
            f"<span style='font-size:14px; font-family:Arial;'>"
            f"<b>📍 DÉPARTEMENT {dep_cible}</b><br>"
            f"📉 Risque : <b>{t_local}%</b> <i style='font-size:11px'>(Nat: {moy_nat_taux}%)</i><br>"
            f"⏳ Survie : <b>{s_local} ans</b> <i style='font-size:11px'>(Nat: {moy_nat_survie} ans)</i>"
            f"</span>"
        ),
        showarrow=False,
        bgcolor="rgba(255, 255, 255, 0.85)",
        bordercolor="gray",
        borderwidth=1,
        align="left"
    )]
)



fig.show(config={'scrollZoom': True})

# --- 5. GÉNÉRATION AUTOMATIQUE DU HTML ---

# Nom du fichier dynamique basé sur le département choisi
nom_fichier = f"analyse_territoriale_{dep_cible}.html"

fig.write_html(
    nom_fichier, 
    config={'scrollZoom': True},
    include_plotlyjs='cdn', 
    full_html=True
)

print(f"✅ Succès ! La carte du département {dep_cible} a été générée : {nom_fichier}")